In [1]:
import json
import random
from pathlib import Path
import collections

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())
# from urllib.parse import quote_plus

In [2]:
#Select granularity to get the data
REGION_FIELDNAME = 'detecteddistrict' #detecteddistrict - for district; detectedcity - for city

#OUTPUT filename region specifier
REGION_OUTPUT_FILE_REG_SPECCIFIER='district'

#what data to extract
DATA_TYPE = 'confirmed' #'confirmed' - default; 'Hospitalized'; 'Recovered'; 'Deceased'

In [3]:
from datetime import date

today = date.today()

# dd/mm/YY
current_date = today.strftime("%Y%m%d")
print("d1 =", current_date)

d1 = 20200420


In [ ]:
import urllib.request, json
from urllib.error import HTTPError

# def get_raw_data(raw_web_url="https://api.covid19india.org/raw_data.json"):
#     with urllib.request.urlopen(raw_web_url) as url:
#         data_dict = json.loads(url.read().decode())
#         return data_dict["raw_data"]


def get_stats_history(stats_history_url="https://api.rootnet.in/covid19-in/stats/history"):
    try:
        with urllib.request.urlopen(stats_history_url) as url:
            data_dict = json.loads(url.read().decode())
            return data_dict
    except HTTPError as e:
        print(f"Using local backup of {stats_history_url}")
        with open("history.json") as f:
            return json.loads(f.read())

def get_stats_history_district(stats_history_url="https://api.covid19india.org/raw_data.json"):
    try:
        with urllib.request.urlopen(stats_history_url) as url:
            data_dict = json.loads(url.read().decode())
            return data_dict
    except HTTPError as e:
        print(f"Using local backup of {stats_history_url}")
        with open("history.json") as f:
            return json.loads(f.read())
        
# def get_state_data(
#     case_count_url="https://api.covid19india.org/state_district_wise.json",
# ):
#     with urllib.request.urlopen(case_count_url) as url:
#         data_dict = json.loads(url.read().decode())
#         return data_dict


# def get_case_count(data, state="Karnataka"):
#     df = pd.DataFrame(data[state]["districtData"])
#     df = df.transpose()
#     df.reset_index(inplace=True)
#     df.rename(columns={"confirmed": "CaseCount", "index": "District"}, inplace=True)
#     df.drop(columns=["lastupdatedtime"], inplace=True)
#     return df

In [ ]:
from typing import List, Dict
#stats:List = get_stats_history()["data"]
stats:List = get_stats_history_district()["raw_data"]

In [ ]:
if DATA_TYPE == 'confirmed':
    district_df = pd.DataFrame([(i[REGION_FIELDNAME],i['dateannounced']) for i in stats])
else:
    district_df = pd.DataFrame([(i[REGION_FIELDNAME],i['statuschangedate']) for i in stats if i['currentstatus']==DATA_TYPE])
# column names in JHU format
REGION = 'Province/State'
district_df.columns = [REGION,'date']

In [ ]:
district_df[district_df['Province/State']=='Bengaluru']

In [ ]:
import pandas as pd
from datetime import datetime
date_list = pd.date_range(start="2020-01-22",end=datetime.today()).strftime("%d/%m/%Y")

In [ ]:
# district_df_pivot = pd.pivot_table(district_df,values=REGION,index=[REGION],columns=['date'],aggfunc=np.count_nonzero,fill_value=0)
district_df_pivot = pd.pivot_table(district_df,values=REGION,index=[REGION],columns=['date'],aggfunc={REGION: np.count_nonzero},fill_value=0)
district_df_pivot_dated = district_df_pivot.reindex(date_list, axis=1).fillna(0)
district_df_final = district_df_pivot_dated.cumsum(axis=1)

## Transform to the JHU data format
See this [Github link to data](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv) for reference

In [ ]:
district_df_final.insert(0, "Country/Region", "India")
district_df_final.insert(1, "Lat", 20)
district_df_final.insert(2, "Long", 70)

In [ ]:
for date in date_list:
    datenew = datetime.strptime(date, "%d/%m/%Y")
    datenew = datetime.strftime(datenew, "%-m/%-d/%y")
    district_df_final = district_df_final.rename(columns = {date:datenew})

In [ ]:
district_df_final.head()

In [ ]:
fname = f'../data/time_series_covid19_{DATA_TYPE}_India_{REGION_OUTPUT_FILE_REG_SPECCIFIER}_{current_date}.csv'
district_df_final.to_csv(fname, header=True)

In [ ]:
!cat $fname

In [ ]:
fname